In [80]:
# Import appropriate modules
import pandas as pd
import numpy as np
from sklearn import preprocessing 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from mlxtend.frequent_patterns import apriori, association_rules 
import warnings
warnings.filterwarnings("ignore")

In [81]:
# Import Data set
df = pd.read_csv("C:\\Users\\dalvi\\Downloads\\Megastore_Dataset_Task_3 3.csv", na_values='.')
num_rows, num_columns = df.shape
df = pd.DataFrame(df)
df_copy = df.copy

# Describe the general characteristcs of the inital dataset (e.g., rows, columns)
print(f"The dataset contains {num_rows} rows and {num_columns} columns.")

The dataset contains 8234 rows and 14 columns.


In [82]:
# Stripping extra spaces in the product name 
df['ProductName'] = df['ProductName'].str.strip() 


In [83]:
# Cast ProductName as string
df = df.astype({"ProductName":'str'})


In [ ]:
# Categorical Variable examination
nominal = ['Region', 'PaymentMethod']
ordinal = ['CustomerOrderSatisfaction', 'OrderPriority']
df[nominal]
df[ordinal]

,Region,PaymentMethod
0,Northeast,Credit Card
1,Northeast,Credit Card
2,Northeast,Credit Card
3,Northeast,Credit Card
4,Northeast,Credit Card
...,...,...
8229,Southeast,PayPal
8230,Southeast,PayPal
8231,Southeast,Credit Card
8232,Southeast,Credit Card


In [ ]:
# Label encode Categorical Variables
#nominal = ['Region', 'PaymentMethod']

label_encoder = preprocessing.LabelEncoder()

# Encode labels in nominal categorical variables
df['Region_encoded'] = label_encoder.fit_transform(df['Region'])
df['Region_encoded']

df['PaymentMethod_encoded'] = label_encoder.fit_transform(df['PaymentMethod'])
df['PaymentMethod_encoded']


0       0
1       0
2       0
3       0
4       0
       ..
8229    1
8230    1
8231    1
8232    1
8233    1
Name: Region_encoded, Length: 8234, dtype: int32

In [86]:
# Ordinal encode ordinal categorical variables
# ordinal = ['CustomerOrderSatisfaction', 'OrderPriority']

# Priority Encoder
priority_encoder = OrdinalEncoder(categories=[['Medium', 'High']])
df['OrderPriority_encoded'] = priority_encoder.fit_transform(df[['OrderPriority']])

# CustomerOrderSatisfaction Encoder
satisfaction_encoder = OrdinalEncoder(categories=[['Prefer not to answer', 'Dissatisfied', 'Very Dissatisfied', 'Satisfied', 'Very Satisfied']])
df['CustomerOrderSatisfaction_encoded'] = satisfaction_encoder.fit_transform(df[['CustomerOrderSatisfaction']])

print(df)



print(df)

      OrderID                        ProductName  Quantity      InvoiceDate  \
0      536370         INFLATABLE POLITICAL GLOBE        48   12/1/2010 8:45   
1      536370      SET2 RED RETROSPOT TEA TOWELS        18   12/1/2010 8:45   
2      536370    PANDA AND BUNNIES STICKER SHEET        12   12/1/2010 8:45   
3      536370      RED TOADSTOOL LED NIGHT LIGHT        24   12/1/2010 8:45   
4      536370  VINTAGE HEADS AND TAILS CARD GAME        24   12/1/2010 8:45   
...       ...                                ...       ...              ...   
8229   580263         DANISH ROSE DELUXE COASTER        12  12/2/2011 12:43   
8230   580263               CACTI TLIGHT CANDLES        16  12/2/2011 12:43   
8231   581316       RED RETROSPOT SUGAR JAM BOWL         1  12/8/2011 11:46   
8232   581316        GLASS  SONGBIRD STORAGE JAR         1  12/8/2011 11:46   
8233   581316           REGENCY SUGAR BOWL GREEN         1  12/8/2011 11:46   

      UnitPrice  TotalCost        Country DiscountA

In [87]:
# Basket 
basket = (df[df['Country'] =="United States"] 
          .groupby(['OrderID', 'ProductName'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('OrderID')) 

basket

ProductName,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR TLIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC TLIGHT HOLDER STAR LARGE,ZINC TLIGHT HOLDER STARS SMALL
OrderID,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
# Defining the hot encoding function to make the data suitable 
# for the concerned libraries 
def hot_encode(x): 
	if(x<= 0): 
		return 0
	if(x>= 1): 
		return 1

# Encoding the datasets 
basket_encoded = basket.applymap(hot_encode) 
basket = basket_encoded 
basket

ProductName,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR TLIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC TLIGHT HOLDER STAR LARGE,ZINC TLIGHT HOLDER STARS SMALL
OrderID,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581279,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
# Apriori Algorithm / Association Rule Creation

itemsets = apriori(basket, min_support = 0.01, use_colnames = True) 
rules = association_rules(itemsets, metric ="lift", min_threshold = 1.0, num_itemsets=2) 

rules_summary = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
print(rules_summary) 

top_rules = rules_summary.nlargest(3, 'lift')
print("Top 3 relevant rules")
print(top_rules)


                                antecedents  \
0                (POPPYS PLAYHOUSE KITCHEN)   
1                  (10 COLOUR SPACEBOY PEN)   
2      (ROUND SNACK BOXES SET OF4 WOODLAND)   
3                  (10 COLOUR SPACEBOY PEN)   
4                  (SPACEBOY BIRTHDAY CARD)   
...                                     ...   
85243        (PACK OF 6 SKULL PAPER PLATES)   
85244        (SET6 RED SPOTTY PAPER PLATES)   
85245       (SET OF 9 BLACK SKULL BALLOONS)   
85246          (SET6 RED SPOTTY PAPER CUPS)   
85247      (PACK OF 20 SKULL PAPER NAPKINS)   

                                             consequents   support  \
0                               (10 COLOUR SPACEBOY PEN)  0.011338   
1                             (POPPYS PLAYHOUSE KITCHEN)  0.011338   
2                               (10 COLOUR SPACEBOY PEN)  0.011338   
3                   (ROUND SNACK BOXES SET OF4 WOODLAND)  0.011338   
4                               (10 COLOUR SPACEBOY PEN)  0.015873   
...            

In [90]:
# Single Transaction example

df_transaction = df[df['OrderID'] == 536370]
print(df_transaction)

    OrderID                         ProductName  Quantity     InvoiceDate  \
0    536370          INFLATABLE POLITICAL GLOBE        48  12/1/2010 8:45   
1    536370       SET2 RED RETROSPOT TEA TOWELS        18  12/1/2010 8:45   
2    536370     PANDA AND BUNNIES STICKER SHEET        12  12/1/2010 8:45   
3    536370       RED TOADSTOOL LED NIGHT LIGHT        24  12/1/2010 8:45   
4    536370   VINTAGE HEADS AND TAILS CARD GAME        24  12/1/2010 8:45   
5    536370                     STARS GIFT TAPE        24  12/1/2010 8:45   
6    536370      VINTAGE SEASIDE JIGSAW PUZZLES        12  12/1/2010 8:45   
7    536370  ROUND SNACK BOXES SET OF4 WOODLAND        24  12/1/2010 8:45   
8    536370              MINI PAINT SET VINTAGE        36  12/1/2010 8:45   
9    536370           MINI JIGSAW CIRCUS PARADE        24  12/1/2010 8:45   
10   536370                MINI JIGSAW SPACEBOY        24  12/1/2010 8:45   
11   536370                  SPACEBOY LUNCH BOX        24  12/1/2010 8:45   

In [ ]:
# Write to CSV
basket.to_csv('C:\\Users\\dalvi\\Desktop\\test\\Allias_Megastore_transactionset.csv')
df.to_csv('C:\\Users\\dalvi\\Desktop\\test\\Allias_Megastore_dataset_cleaned.csv')

